In [3]:
import numpy as np
import cv2
 
# Cargamos la imagen
original = cv2.imread("papa20.png")
cv2.imshow("original", original)

# Convertimos a escala de grises 
gris = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
cv2.imshow("gris", gris)
 
# Aplicar suavizado Gaussiano para eliminación de ruido
gauss = cv2.GaussianBlur(gris, (5,5), 0)
cv2.imshow("suavizado", gauss)

# Umbralización
a, th = cv2.threshold(gauss,70,210,cv2.THRESH_BINARY)# + cv2.THRESH_OTSU)
gaussth=cv2.Canny(th, 10, 180)
cv2.imshow("binarizado", gaussth)

mascara = cv2.bitwise_and(gaussth, gaussth, mask= th)
cv2.imshow("mask",mascara)

#Transformada de Hough
"""
circles = cv2.HoughCircles(gaussth,cv2.HOUGH_GRADIENT,3,10,
                            param1=20,param2=50,minRadius=50,maxRadius=120)
circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # Dibuja la circusnferencia del círculo
    cv2.circle(gaussth,(i[0],i[1]),i[2],(0,255,0),2)
    # dibuja el centro del círculo
    cv2.circle(gaussth,(i[0],i[1]),2,(0,0,255),3)
cv2.imshow('círculos detectados',gaussth)
"""

# Detectamos los bordes con Canny
canny = cv2.Canny(gaussth, 50, 150)
cv2.imshow("canny", canny)

# Dilatación y Erosión de los contornos
canny = cv2.dilate(canny, None, iterations=1)
canny =cv2.erode(canny, None, iterations= 1)
#cv2.imshow("dilatación y erosión", canny)

#opening
kernel=np.ones((5,5),np.uint8)
opening_op= cv2.morphologyEx(canny,cv2.MORPH_OPEN,kernel)
closing_op= cv2.morphologyEx(canny,cv2.MORPH_CLOSE,kernel)
cv2.imshow("opening", canny)

# Buscamos los contornos
(contornos,_) = cv2.findContours(canny.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

# Mostramos el número de contornos encontrados
print("Se encontró {} objetos".format(len(contornos)))

cv2.drawContours(original,contornos,-1,(0,0,255), 2)
cv2.imshow("contornos", original)

#Calculo de pixeles por contorno
permon = round(cv2.arcLength(contornos[1], True),2)
perpapa = round(cv2.arcLength(contornos[3], True),2)
porcentaje1 = round(permon/perpapa*100, 2)
print(f'El perimetro de la moneda es: {permon} pixeles')
print(f'El perimetro de la papa es: {perpapa} pixeles')
print(f'La proporción del perimetro de la ficha en relación a la papa es:{porcentaje1}%')

"""
listpx =[]
for i in range (len(contornos)):
    cv2.drawContours(gauss, contornos, i, (0,255,0),3)
    print("Contorno", "=", len(contornos[i]), "px")
    listpx.append(len(contornos[i]))
#print(listpx)
"""
#Calculo del área de los 2 contornos
areamon = cv2.contourArea(contornos[1])
areapapa = cv2.contourArea(contornos[3])
porcentaje2 = round(areamon/areapapa*100, 2)
print(f'El área de la moneda es: {areamon} pixeles cuadrados')
print(f'El área de la papa es: {areapapa} pixeles cuadrados')
print(f'La proporción del área de la ficha en relación a la papa es: {porcentaje2}%')

cv2.waitKey(0)

Se encontró 4 objetos
El perimetro de la moneda es: 467.83 pixeles
El perimetro de la papa es: 1967.42 pixeles
La proporción del perimetro de la ficha en relación a la papa es:23.78%
El área de la moneda es: 14917.5 pixeles cuadrados
El área de la papa es: 246415.0 pixeles cuadrados
La proporción del área de la ficha en relación a la papa es: 6.05%


-1